In [1]:
import Graph_Amazon_Movies as gam
import Amazon_Movie_Parser as amp
import numpy as np
from networkx.algorithms import bipartite
import networkx as nx
import Amazon_Movie_Parser as prs
import datetime
from numpy import savetxt

In [2]:
start_time = datetime.datetime.now()
grp = gam.Graph_Amazon()
max_connected_gr_amazon_movies = grp.Create_Bipartite(file_name = 'data/movies.txt', \
                                                      n_movies = 2000, prs_out = 'dictionary')
end_time = datetime.datetime.now()
print('Calculation time: {}'.format(end_time-start_time))

Calculation time: 0:00:00.136464


In [3]:
bottom_nodes, top_nodes = bipartite.sets(max_connected_gr_amazon_movies)

In [4]:
# Data check
# some MOVIES don't start with B but instead full numbers like: 0790747324
print('max_connected_gr_amazon_movies:')
count_B = 0
for movie in top_nodes:
    if movie.startswith('B'):
        count_B += 1

print('Movies: {} ... starts with B: {}'.format(len(top_nodes), count_B))

count_A=0
for movie in bottom_nodes:
    if movie.startswith('A'):
        count_A += 1

print('Users: {} ... starts with A: {}'.format(len(bottom_nodes), count_A))

grp.Is_Connected_Bipartite(max_connected_gr_amazon_movies)
# edges between movies
#max_grp_top = bipartite.projected_graph(max_connected_gr_amazon_movies, grp.top_nodes)
# shows all the records BEFORE picking up the MAX CONNECTED component - most likely it will be a DISCONNECTED GRAPH
# returns a LIST
#edge_list_ALL = grp.Get_All_Edges_with_Weight(grp.gr_amazon_movies)
#print(edge_list_ALL[0:5])
#edge_list_connected = grp.Get_All_Edges_with_Weight(max_connected_gr_amazon_movies)
#list(max_connected_gr_amazon_movies.edges)[0:5]
# shows the nodes of the max connected component/subgraph
#grp.Show_Nodes()
#if your graph is too BIG, it is better not to draw it, it takes time..
#grp.Draw_Bipartite(max_connected_gr_amazon_movies)
#shows the edges with weights
#list(max_connected_gr_amazon_movies.edges.data('weight', default=1))[0:5]

max_connected_gr_amazon_movies:
Movies: 20 ... starts with B: 15
Users: 1832 ... starts with A: 1832
Is connected: True ... Is bipartite: True


In [5]:
#saves the movies/graph in the format of:
'''
A141HP4LYPW,B003AI2VGA,3.0
A141HP4LYPW,B000063W1R,4.0
A141HP4LYPW,6304286961,5.0
'''
#max_connected_gr_amazon_movies
file_name_saved = amp.Save_Movies(max_connected_gr_amazon_movies)

Graph info saved in: Movies_Connected_1885_11.5.2020_21.57.36.txt


In [6]:
# returns a list in the format of:
# [['userId', 'productId', 'score'], ['A141HP4LYPW', 'B003AI2VGA', '3.0']]
#the_giant_component_read = amp.Read_Connected_Movies("Movies_Connected_4643_11.5.2020_21.11.17.txt")
the_giant_component_read = amp.Read_Connected_Movies(file_name_saved)
#test[0].split(sep=",")

In [7]:
# it is mostly for a double check to see if the movies were saved to the file properly 
#and check if we read them and create a graph it will still be a connected graph or not
grp_giant = grp.Create_Graph_From_List_WITH_Weight(the_giant_component_read)

In [8]:
# double check if the numbers are matching before saving the records
# and after creating the graph from the saved file
print('BEFORE: # of nodes: {} ... # of edges: {}'.format(len(max_connected_gr_amazon_movies.nodes), len(max_connected_gr_amazon_movies.edges)))
print('AFTER: # of nodes: {} ... # of edges: {}'.format(len(grp_giant.nodes), len(grp_giant.edges)))

BEFORE: # of nodes: 1852 ... # of edges: 1885
AFTER: # of nodes: 1852 ... # of edges: 1885


In [9]:
# here is the check of the graph read
grp.Is_Connected_Bipartite(grp_giant)

Is connected: True ... Is bipartite: True


In [10]:
# this is to fetch all nodes without edges, so that we can use them to predict the edges
grp_giant_no_edges = grp.Create_Graph_From_List_NO_EDGE(grp_giant)
#if you check it, you'll see no edges but nodes
#grp_giant_no_edges.edges
#list(grp_giant_no_weights.nodes)[0:5]
#grp.Is_Connected_Bipartite(grp_giant_no_weights)

In [ ]:
start_time = datetime.datetime.now()
df_movies = amp.Reorganize_Edges_DataFrame(max_connected_gr_amazon_movies, input_type = 'Graph')

node_movies = df_movies['movieId'].unique()
node_users = df_movies['userId'].unique()
nodes = np.concatenate((node_users, node_movies), axis=0)
n_nodes = len(nodes)
#n_nodes = len(max_connected_gr_amazon_movies.nodes)
print('# of nodes in the giant connected component: {}'.format(n_nodes))
# Probability matrix, if there is an edge between node[i] and node[j], 
# in other words, if a user reviewed a movie with a score from 1 to 5, P[i][j]=score[ij]
# then the row will be normalized to have the values between 0 and 1
#the weight will be the

P = np.zeros((n_nodes, n_nodes))
R = np.zeros((n_nodes, n_nodes))

i = 0
for n_r in nodes:
    j = 0
    for n_c in nodes:
        edge_index = df_movies.query("movieId == '" + n_r + "' and userId == '" + n_c + "'").index
        if len(edge_index) != 0:
            P[i][j] = df_movies.iloc[edge_index[0], 2]
            P[j][i] = P[i][j]
            #print('i={} .. j={} .. n_r={} .. n_c={}'.format(i, j, n_r, n_c))
        j += 1
    print('node-{}: {} is checked out of {} nodes.'.format(i, n_r, n_nodes))
    i += 1
    
end_time = datetime.datetime.now()
print('Calculation time: {}'.format(end_time-start_time))

Graph has been reorganized as a DataFrame, row format: ['userId', 'movieId', 'score']
# of nodes in the giant connected component: 1852
node-0: B003AI2VGA is checked out of 1852 nodes.
node-1: B000063W1R is checked out of 1852 nodes.
node-2: B0001G6PZC is checked out of 1852 nodes.
node-3: B0016OLXN2 is checked out of 1852 nodes.
node-4: B00004CQT3 is checked out of 1852 nodes.
node-5: B002OHDRF2 is checked out of 1852 nodes.
node-6: B0078V2LCY is checked out of 1852 nodes.
node-7: B0092MDYV4 is checked out of 1852 nodes.
node-8: B000CDSS86 is checked out of 1852 nodes.
node-9: 0800103688 is checked out of 1852 nodes.
node-10: 0790747324 is checked out of 1852 nodes.
node-11: B000NDFLWG is checked out of 1852 nodes.
node-12: B008FPU7AA is checked out of 1852 nodes.
node-13: 6300157423 is checked out of 1852 nodes.
node-14: B00004CQTF is checked out of 1852 nodes.
node-15: B000IZJR3I is checked out of 1852 nodes.
node-16: 630436119X is checked out of 1852 nodes.
node-17: B001GBPZRU is c

node-158: A3SPJY549DX is checked out of 1852 nodes.
node-159: A2RW7HXNEJ0 is checked out of 1852 nodes.
node-160: A4OWXQLYVER is checked out of 1852 nodes.
node-161: AG9A3ZL7ODL is checked out of 1852 nodes.
node-162: A2XTMDDO0UC is checked out of 1852 nodes.
node-163: A3GV24GH1AF is checked out of 1852 nodes.
node-164: A1VT9SADFD9 is checked out of 1852 nodes.
node-165: A31VEVBN7K8 is checked out of 1852 nodes.
node-166: A1O05HYBO2L is checked out of 1852 nodes.
node-167: A1UMSB7LAW0 is checked out of 1852 nodes.
node-168: A22QDGL361U is checked out of 1852 nodes.
node-169: A27GP310AJR is checked out of 1852 nodes.
node-170: A39CX0EE4BZ is checked out of 1852 nodes.
node-171: A30PZUE2YQH is checked out of 1852 nodes.
node-172: A91VKBOACYQ is checked out of 1852 nodes.
node-173: A3VC1CIRFDX is checked out of 1852 nodes.
node-174: A2PKRPEODLQ is checked out of 1852 nodes.
node-175: A3A4WQL80WO is checked out of 1852 nodes.
node-176: A273SGLDMWW is checked out of 1852 nodes.
node-177: A1

In [ ]:
def Normalize_Array(arr_to_normalize):
    arr_sum_per_row = arr_to_normalize.sum(axis=1)
    arr_norm = arr_to_normalize/np.tile(arr_sum_per_row, (arr_sum_per_row.shape[0], 1)).T
    return arr_norm

In [ ]:
# until roun(, 3) >> it's ok. when we round to 4, then it messes up..
# even with round(3) >> if we save in a file, it messes up..
P_norm = np.round(Normalize_Array(P), 3)
#P_norm = Normalize_Array(P)

In [ ]:
P_norm

In [ ]:
# SUM-check
#print('SUM-check')
#P_norm.sum(axis=1)

In [ ]:
savetxt(amp.FileNameUnique(prefix = "P_norm_", suffix = '.csv'), P_norm, delimiter=',')